In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome 
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.keys import Keys
import time 
from datetime import datetime, timedelta
import os 
import uuid 
import json 


class News_Scraper:

    def __init__(self, url) -> None:
        # url to be parameter entered in __name__ == "__main__"
        self.url = url 
        
        #Creates an empty list for links to each article (to be collected from main news page).
        self.href_list = []
        
        #Creates a dictionary of empty lists to store information about each article. 
        self.article_data = {
            "title": [],
            "author": [],
            "date": [],
            "source": [],
            #"uuid": []
        }

        self.store_info = "./raw_data"

        # Using ChromeDriver / Selenium to open webpage 
        self.driver = Chrome(ChromeDriverManager().install())
        self.driver.get(url)
        # time.sleep(2)

        # Accepts cookies if they are there. If not pass.
        try:
            click_accept_cookies = self.driver.find_element(By.ID, "cookie-accept-link-text")
            click_accept_cookies.click()
            # time.sleep(2)

            # Once cookies are accepted. Scroll to the bottom of the initial page.
            try:
                scroll_to_bottom = self.driver.find_element(By.TAG_NAME, 'body') 
                scroll_to_bottom.send_keys(Keys.END)
                # time.sleep(2)
            except:
                print("Didn't scroll to Medical Links")
                pass 

        except:
            print("Couldn't accept cookies, or there were none to accept")
            pass 
        return

    
    def news_button(self):
        """
        Method to go to the 'news' page where the articles are listed
        """
        # Below no longer working (had been working before 14/05/22 - WHY???)
        # try:
        #     time.sleep(2)
        #     click_news_button = self.driver.find_element(By.LINK_TEXT, "News")
        #     click_news_button.click()
        #     time.sleep(2)
        # except:
        #     print("Didn't click 'News' button")
        #     pass   
        
        try:
            # news_link_container = self.driver.find_element(By.XPATH, '//div[@class="col-xs-6 col-sm-3 footer-menu life-sciences-useful-links"]')
            # news_link = self.driver.find_element(By.XPATH, f"//a[contains (@href, '/life-sciences/news')]")
            # news_link.click 
            self.driver.get("https://www.news-medical.net/medical/news")
        except:
            print("Didn't click on news link")
            pass


    def link_list(self):
        """
        Return a list storing the link for each article on the page.
        This gives a list everytime, and the next list just adds on the previous 
        As each article doesn't have a clear unique identifier (i.e. product number). The URL will be the unique identifier 
        """
        container = self.driver.find_element(By.XPATH, '//div[@class="posts publishables-list-wrap first-item-larger"]')
        self.elements = container.find_elements(By.XPATH, ".//a[@href]")
        for element in self.elements:  
            self.href = element.get_attribute('href') # this is returnning as the same as url for the news page. NOT the individual hrefs for each article as aimed. WHY????
            self.href_list.append(self.href) # This returns a list of all the urls for each article on the news page. 
        
        print(f"self.href is {self.href} & is type {type(self.href)}") 
        print(f"first eleement of self.href_list is {self.href_list[0]} & is type {type(self.href_list)} & is length {len(self.href_list)}")
        
        self.title_list = []
        self.author_list = []
        self.date_list = []
        self.source_list = []
        self.href_list_shorten = self.href_list[:20]
        self.uuid_list = []

        # link_ID = uuid.uuid4()

        # for index, element in enumerate(self.href_list_shorten):
        #     if (index + 1 < len(self.href_list_shorten) and index - 1 >= 0):
        #         prev_element = str(self.href_list_shorten[index - 1])
        #         curr_element = str(element)
        #         next_element = str(self.href_list_shorten[index + 1])
        #         print(f"Previous is: {prev_element}, current is {curr_element}, and next is {next_element}")
            

        for i in range(len(self.href_list_shorten)):

            item = str(self.href_list_shorten[i])

            if i >= 1:
                
                if (item != str(self.href_list_shorten[0])) and (item != str(self.href_list_shorten[i-1])):
                    self.driver.get(item)
                    self.title = self.driver.title
                    self.title_list.append(self.title)
                    
                    self.author_container = self.driver.find_element(By.CLASS_NAME, "article-meta-contents")
                    self.author_link = self.author_container.find_element(By.XPATH, ".//a[@href]")
                    self.author = self.author_link.text
                    self.author_list.append(self.author)
                    
                    self.date = self.driver.find_element(By.CLASS_NAME, "article-meta-date").text
                    self.date_list.append(self.date)

                    # self.uuid_list.append(link_ID)
                    self.ID_for_each_article()
                    
                    
                else:
                    pass
            else:
                pass

        print(f"Title_list is: {self.title_list} & has type {type(self.title_list)} and length {len(self.title_list)})")
        print(f"Author list is {self.author_list} & has type {type(self.author_list)} and length {len(self.author_list)})")
        print(f"Date list is {self.date_list} & has type {type(self.date_list)} and length {len(self.date_list)})")
        print(f"uuid_list is: {self.uuid_list} & has type {type(self.uuid_list)} and length {len(self.uuid_list)}")

    def ID_for_each_article(self):
        """ 
        Create a list of uuid for each item in a list (i.e. for each url in the link_list, create a associated uuid
        Using version 4 
        """
        # self.article_data["uuid"]
        # self.uuid_list = []
        link_ID = uuid.uuid4()
        self.uuid_list.append(link_ID)
        # print(f"uuid_list is: {self.uuid_list}")
        # return self.uuid_list

if __name__ == "__main__":
    news_scraper = News_Scraper('https://www.news-medical.net/')
    # news_scraper.scroll_down()
    news_scraper.news_button()
    news_scraper.link_list()
    



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/frankiejames/.wdm/drivers/chromedriver/mac64_m1/102.0.5005.61/chromedriver] found in cache
/var/folders/sf/lnrr_g5s6bd9cf8zjnzr2b6w0000gn/T/ipykernel_12022/3430883776.py:35: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = Chrome(ChromeDriverManager().install())


self.href is https://www.news-medical.net/medical/news?page=2 & is type <class 'str'>
first eleement of self.href_list is https://www.news-medical.net/news/20220608/Einstein-researchers-receive-2435-million-NIH-grant-to-study-the-effects-of-SARS-CoV-2-on-the-brain.aspx & is type <class 'list'> & is length 45
Title_list is: ['Black, Hispanic patients with advanced liver cancer have lower chance of receiving immunotherapy', 'Vegan diet leads to decreased weight and improved insulin sensitivity', 'Individuals who had COVID‐19 report more cognitive failures at work, study shows', "Fathers’ depression in economically struggling families may be damaging to the couple's relationship", 'Childhood cancer survivors 80% more likely to be undertreated for cardiovascular risk factors', 'Early identification and treatment can reduce financial costs associated with spinal muscular atrophy', 'Women with PCOS more likely to be diagnosed with additional medical conditions, study shows', 'Walking for exe